In [ ]:
import numpy as np
import pymc as pm

import xarray as xr

from pymc_marketing.clv.models.beta_geo import BetaGeoModel
from pymc_marketing.clv.distributions import continuous_contractual

from lifetimes.fitters.beta_geo_fitter import BetaGeoFitter

from scipy.special import expit, hyp2f1

In [ ]:
rng = np.random.default_rng(34)

a_true = 0.8
b_true = 2.5
alpha_true = 3
r_true = 4

# Number of subjects
N = 500
# Subject level parameters
p = pm.draw(pm.Beta.dist(a_true, b_true, size=N), random_seed=rng)
lam = pm.draw(pm.Gamma.dist(r_true, alpha_true, size=N), random_seed=rng)

T = pm.draw(pm.DiscreteUniform.dist(lower=20, upper=40, size=N), random_seed=rng)

data = continuous_contractual.rng_fn(rng, lam, p, T, 0, size=N)

recency = data[..., 0]
frequency = data[..., 1]
alive = 1 - data[..., 2]

In [ ]:
model = BetaGeoModel(
    customer_id=np.arange(N),
    frequency=frequency,
    recency=recency,
    T=T,
)

In [ ]:
model.fit(chains=1);

In [ ]:
a = xr.DataArray([10, 20, 30], coords={"customer_id": range(3)})
b = xr.DataArray([3, 4], coords={"a": [30, 40]})
c = (b + a)
c.transpose("customer_id", "a").mean(("customer_id", "a"))

In [ ]:
model.expected_probability_alive(np.arange(N), frequency, recency, T)

In [ ]:
def true_expected_number_of_purchases(t):
    left_term = (a_true + b_true - 1) / (a_true - 1)
    right_term = 1 - (alpha_true / (alpha_true + t)) ** r_true * hyp2f1(
        r_true, b_true, a_true + b_true - 1, t / (alpha_true + t)
    )
    return left_term * right_term

In [ ]:
t = np.array(range(20, 40, 2))
true_expected_number_of_purchases(t)